In [10]:
from flask import Flask, request, jsonify
from threading import Thread
import json
import dateutil 


from src.agents.meeting_agent import meeting_agent
from src.agents.priority_agent import get_priority_async
from src.agents.slots_agent import slots_agents

from src.utils.attendees import get_attendees_with_from
from src.utils.interval_handling import get_free_time_slots,get_free_time_slots_priority

In [11]:
import time
import asyncio
from datetime import datetime, timedelta
from dateutil import parser

# Alternative version with even more optimization
async def your_meeting_assistant_ultra_optimized(data): 
    # Start timer for all concurrent operations
    start_total = time.time()

    # Start timer
    start = time.time()
    
    # Create tasks for all independent operations
    priority_task = get_priority_async(data["EmailContent"])
    meeting_agent_task = meeting_agent.run(data["EmailContent"])

    # End timer
    end = time.time()
    
    # Calculate and print runtime
    runtime = end - start
    print(f"meeting_agent Runtime: {runtime:.4f} seconds")
    

    # Start timer
    start = time.time()
    
    # Get attendees synchronously (assuming this is fast)
    attendees = get_attendees_with_from(data)
    print("Extracted Attendees")
    print(attendees)

    # End timer
    end = time.time()
    
    # Calculate and print runtime
    runtime = end - start
    print(f"get_attendees_with_from Runtime: {runtime:.4f} seconds")
    
    # Wait for priority and meeting agent results
    priority_result, meeting_agent_results = await asyncio.gather(
        priority_task,
        meeting_agent_task
    )
    
    # Process results
    priority = priority_result.data.priority
    data["Priority"] = priority
    
    response_data_meeting = {
        "start_timeframe": meeting_agent_results.data.start_timeframe,
        "end_timeframe": meeting_agent_results.data.end_timeframe,
        "duration_minutes": meeting_agent_results.data.duration_minutes,
        "success": True,
        "message": "Successfully extracted meeting timeframes"
    }
    
    print("EXTRACTED TIME ELEMENTS")
    print(response_data_meeting)
    
    # Get free slots
    free_slots, user_schedule_dict = get_free_time_slots(
        attendees, 
        response_data_meeting["duration_minutes"], 
        response_data_meeting["start_timeframe"], 
        response_data_meeting["end_timeframe"]
    )

    if len(free_slots)>0:
        
    
    else :
        print('No available time slots found for the specified duration\n ')
        free_slots = get_free_time_slots_priority(
            attendees, 
            response_data_meeting["duration_minutes"], 
            response_data_meeting["start_timeframe"], 
            response_data_meeting["end_timeframe"], 
            priority, 
            len(data["Attendees"])
        )
    print(free_slots)

    # Get slots suggestion
    slots_agent_results = await slots_agents.run(
        f"Free Slots {free_slots} Duration: {response_data_meeting['duration_minutes']}"
    )
    
    response_data_slots = {
        "start_time_of_meeting": slots_agent_results.data.start_time_of_meeting,
        "end_time_of_meeting": slots_agent_results.data.end_time_of_meeting,
        "duration_minutes": slots_agent_results.data.duration_minutes,
        "metadata": slots_agent_results.data.metadata
    }
    
    print("Suggested Slot")
    print(response_data_slots)
    
    print("After Processing\n")
    # Parse the start time string to datetime object
    start_time_dt = dateutil.parser.isoparse(slots_agent_results.data.start_time_of_meeting)
    
    # Add the duration
    end_time_dt = start_time_dt + timedelta(minutes=int(slots_agent_results.data.duration_minutes))
    
    # Convert back to ISO format string
    end_time_str = end_time_dt.isoformat()

    response_data_slots['end_time_of_meeting'] = str(end_time_str)

    print(response_data_slots)

    # Parse the response
    raw = response_data_slots["start_time_of_meeting"].strip()
    try:
        dt = parser.isoparse(raw)
    except Exception:
        dt = parser.parse(raw) 
    
    available_start_time = dt.strftime("%Y-%m-%dT%H:%M:%S%z")
    available_end_dt = dt + timedelta(minutes=response_data_meeting["duration_minutes"])
    available_end_time = available_end_dt.strftime("%Y-%m-%dT%H:%M:%S%z")

    # End total timer
    end_total = time.time()
    total_runtime = end_total - start_total
    print(f"Total optimized runtime: {total_runtime:.4f} seconds")

    # Build output efficiently
    output = {
        "Request_id": data["Request_id"],
        "Datetime": (datetime.strptime(data["Datetime"], "%d-%m-%YT%H:%M:%S")).strftime("%Y-%m-%dT%H:%M:%S"),
        "Location": data["Location"],
        "From": data["From"],
        "Attendees": [
            {**attendee, "events": user_schedule_dict[attendee["email"]]} 
            for attendee in data["Attendees"]
        ],
        "Subject": data["Subject"],
        "EmailContent": data["EmailContent"],
        "EventStart": available_start_time,
        "EventEnd": available_end_time,
        "Duration_mins": str(response_data_meeting["duration_minutes"]),
        "MetaData": response_data_slots['metadata']
    }
    
    return output

In [9]:
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Client Feedback",
    "EmailContent": "Hi Team. We’ve received the final feedback from the client. Let’s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M."
}
await your_meeting_assistant_ultra_optimized(data)

meeting_agent Runtime: 0.0000 seconds
Extracted Attendees
['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']
get_attendees_with_from Runtime: 0.0000 seconds


/tmp/ipykernel_20548/3285486962.py:48: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  priority = priority_result.data.priority
/tmp/ipykernel_20548/3285486962.py:52: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_timeframe": meeting_agent_results.data.start_timeframe,
/tmp/ipykernel_20548/3285486962.py:53: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_timeframe": meeting_agent_results.data.end_timeframe,
/tmp/ipykernel_20548/3285486962.py:54: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": meeting_agent_results.data.duration_minutes,


EXTRACTED TIME ELEMENTS
{'start_timeframe': '2025-07-23T10:00:00+05:30', 'end_timeframe': '2025-07-23T10:59:00+05:30', 'duration_minutes': 60, 'success': True, 'message': 'Successfully extracted meeting timeframes'}
[{'StartTime': '2025-07-23T09:30:00+05:30', 'EndTime': '2025-07-23T10:30:00+05:30', 'NumAttendees': 3, 'Attendees': ['usertwo.amd@gmail.com', 'userone.amd@gmail.com', 'userthree.amd@gmail.com'], 'Summary': 'Client Feedback'}]
[{'StartTime': '2025-07-23T09:30:00+05:30', 'EndTime': '2025-07-23T10:30:00+05:30', 'NumAttendees': 3, 'Attendees': ['usertwo.amd@gmail.com', 'userone.amd@gmail.com', 'userthree.amd@gmail.com'], 'Summary': 'Client Feedback'}, {'StartTime': '2025-07-23T10:00:00+05:30', 'EndTime': '2025-07-23T11:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Client Meeting - IMPORTANT'}]
[{'StartTime': '2025-07-23T09:30:00+05:30', 'EndTime': '2025-07-23T10:30:00+05:30', 'NumAttendees': 3, 'Attendees': ['usertwo.amd@gmail.com', 'userone.amd@gmail.com'

/tmp/ipykernel_20548/3285486962.py:90: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_time_of_meeting": slots_agent_results.data.start_time_of_meeting,
/tmp/ipykernel_20548/3285486962.py:91: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_time_of_meeting": slots_agent_results.data.end_time_of_meeting,
/tmp/ipykernel_20548/3285486962.py:92: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": slots_agent_results.data.duration_minutes,
/tmp/ipykernel_20548/3285486962.py:93: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "metadata": slots_agent_results.data.metadata
/tmp/ipykernel_20548/3285486962.py:101: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  start_time_dt = dateutil.parser.isoparse(slots_agent_results.data.start_time_of_meeting)


TypeError: object of type 'NoneType' has no len()

In [13]:
from flask import Flask, request, jsonify
import asyncio
import json

app = Flask(__name__)
received_data = []

@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    
    # Run async function in sync context
    new_data = asyncio.run(your_meeting_assistant_ultra_optimized(data))

    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.191.103:5000
Press CTRL+C to quit
152.57.113.158 - - [20/Jul/2025 06:23:25] "POST /receive HTTP/1.1" 400 -



 Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Agentic AI Project Status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}
meeting_agent Runtime: 0.0000 seconds
Extracted Attendees
['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']
get_attendees_with_from Runtime: 0.0000 seconds


/tmp/ipykernel_19789/31232433.py:48: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  priority = priority_result.data.priority
/tmp/ipykernel_19789/31232433.py:52: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_timeframe": meeting_agent_results.data.start_timeframe,
/tmp/ipykernel_19789/31232433.py:53: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_timeframe": meeting_agent_results.data.end_timeframe,
/tmp/ipykernel_19789/31232433.py:54: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": meeting_agent_results.data.duration_minutes,


EXTRACTED TIME ELEMENTS
{'start_timeframe': '2025-07-24T00:00:00+05:30', 'end_timeframe': '2025-07-24T23:59:00+05:30', 'duration_minutes': 30, 'success': True, 'message': 'Successfully extracted meeting timeframes'}
[{'StartTime': '2025-07-23T18:00:00+05:30', 'EndTime': '2025-07-24T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-24T10:30:00+05:30', 'EndTime': '2025-07-24T11:00:00+05:30', 'NumAttendees': 3, 'Attendees': ['usertwo.amd@gmail.com', 'userone.amd@gmail.com', 'userthree.amd@gmail.com'], 'Summary': 'Agentic AI Project Status Update'}, {'StartTime': '2025-07-24T18:00:00+05:30', 'EndTime': '2025-07-25T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}]
[{'StartTime': '2025-07-23T18:00:00+05:30', 'EndTime': '2025-07-24T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-24T10:30:00+05:30', 'EndTime': '2025-07-24T11:00:00+05:30', 'NumAtt

...

Suggested Slot
{'start_time_of_meeting': '2025-07-24T09:00:00+05:30', 'end_time_of_meeting': '2025-07-24T09:30:00+05:30', 'duration_minutes': 30, 'metadata': 'The meeting is scheduled in the first available slot which is 30 minutes long.'}
Total optimized runtime: 6.8325 seconds



 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-23T18:00:00+05:30",
          "EndTime": "2025-07-24T09:00:00+05:30",
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-24T10:30:00+05:30",
          "EndTime": "2025-07-24T11:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "usertwo.amd@gmail.com",
            "usero

152.57.113.158 - - [20/Jul/2025 06:24:00] "POST /receive HTTP/1.1" 200 -


Suggested Slot
{'start_time_of_meeting': '2025-07-24T09:00:00+05:30', 'end_time_of_meeting': '2025-07-24T09:30:00+05:30', 'duration_minutes': 30, 'metadata': 'Optimal time slot found from the first available timeframe.'}
Total optimized runtime: 6.6159 seconds



 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-23T18:00:00+05:30",
          "EndTime": "2025-07-24T09:00:00+05:30",
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-24T10:30:00+05:30",
          "EndTime": "2025-07-24T11:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "usertwo.amd@gmail.com",
            "userone.amd@gmail.com",


152.57.113.158 - - [20/Jul/2025 06:24:16] "POST /receive HTTP/1.1" 200 -


Suggested Slot
{'start_time_of_meeting': '2025-07-24T09:00:00+05:30', 'end_time_of_meeting': '2025-07-24T09:30:00+05:30', 'duration_minutes': 30, 'metadata': 'The meeting is scheduled in the first available slot, which is from 9:00 AM to 10:30 AM. The duration is 30 minutes, so the meeting ends at 9:30 AM.'}
Total optimized runtime: 6.9726 seconds



 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-23T18:00:00+05:30",
          "EndTime": "2025-07-24T09:00:00+05:30",
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-24T10:30:00+05:30",
          "EndTime": "2025-07-24T11:00:00+05:30",
          "NumAttendees": 3,
          

In [ ]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [ ]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()